In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
context_size = 5
d_embed = 16

In [3]:
tril = torch.tril(torch.ones(context_size, context_size))
mask = torch.zeros(context_size, context_size)
mask = mask.masked_fill(tril == 0, float('-inf'))
mask = F.softmax(mask, dim=-1)
mask

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])

In [4]:
value = torch.randn(context_size, d_embed)
value

tensor([[-0.7482, -1.0120, -0.0785, -0.5177,  1.1935, -0.0689, -0.7837,  0.7090,
          0.9079, -0.0320,  0.5597,  0.9166,  0.4053,  2.3993, -0.5944,  0.8245],
        [-1.3647, -0.4895, -1.9848, -1.0756, -0.3199,  0.0399,  0.0874, -1.1726,
          1.1920, -0.6296,  0.1530,  2.0978, -2.0740,  1.9029,  0.2107, -1.0366],
        [ 0.3438, -0.8635,  0.5719, -0.0806,  1.1393, -0.6948,  0.1543, -1.6219,
          0.4630, -0.3029,  0.9259, -0.2390, -0.0881,  1.0604, -1.4539,  0.1952],
        [ 0.4775, -1.0906, -0.3581, -0.1567, -0.9973, -0.2457,  0.3069,  1.1748,
         -0.3095, -0.2139, -0.1392,  2.0191,  0.7314, -0.3383,  0.9137, -0.8571],
        [-0.0603, -0.6378,  0.6487,  0.9391,  0.9546,  0.1582, -0.2857, -0.9937,
         -0.6400, -0.0252,  1.4679,  1.1117,  0.0677, -0.2794,  0.8476, -0.0346]])

In [5]:
out = torch.matmul(mask, value)
out

tensor([[-0.7482, -1.0120, -0.0785, -0.5177,  1.1935, -0.0689, -0.7837,  0.7090,
          0.9079, -0.0320,  0.5597,  0.9166,  0.4053,  2.3993, -0.5944,  0.8245],
        [-1.0565, -0.7508, -1.0316, -0.7967,  0.4368, -0.0145, -0.3482, -0.2318,
          1.0499, -0.3308,  0.3563,  1.5072, -0.8344,  2.1511, -0.1919, -0.1061],
        [-0.5897, -0.7884, -0.4971, -0.5580,  0.6710, -0.2413, -0.1807, -0.6952,
          0.8543, -0.3215,  0.5462,  0.9251, -0.5856,  1.7875, -0.6126, -0.0056],
        [-0.3229, -0.8639, -0.4624, -0.4576,  0.2539, -0.2424, -0.0588, -0.2277,
          0.5633, -0.2946,  0.3748,  1.1986, -0.2563,  1.2561, -0.2310, -0.2185],
        [-0.2704, -0.8187, -0.2402, -0.1783,  0.3940, -0.1623, -0.1042, -0.3809,
          0.3227, -0.2407,  0.5935,  1.1812, -0.1915,  0.9490, -0.0153, -0.1817]])

# Why query, key, value?

The model needs a communication step: matmul, concat, linear whatever it is, pass two tokens

concat, linear -> for every step, number of vectors are powered by 2 -> the model size gets enormously big

So it leads to the goal: Input tensor shape and Output tensor shape must be the same.
1. get value vector: it will remain its size
2. do a weighted sum between the value vectors
3. to do weighted sum, we need to know the importance of each value vector: get it by query, key

In [2]:
torch.log(torch.tensor(1/10))

tensor(-2.3026)